# Eye-tracker

# I. Giới thiệu

- Ở người, hầu hết các loài thú có vú và chim, mống mắt (tiếng Anh: Iris, số nhiều: irides hoặc irises) là một cấu trúc mỏng, hình tròn nằm trong mắt, có công dụng điều chỉnh đường kính và kích cỡ của đồng tử, cũng chính là lượng ánh sáng đi đến võng mạc. Màu mắt được quyết định bởi màu của mống mắt. Đồng tử thường được coi là lỗ khẩu, còn mống mắt được coi là khẩu [ref](https://vi.wikipedia.org/wiki/M%E1%BB%91ng_m%E1%BA%AFt)


- Đặc tính:
    + Ổn định - mống mắt của con người được hình thành từ 10 tháng tuổi và không thay đổi trong suốt cuộc đời của mỗi người.

    + Không nhầm lẫn – xác xuất hai mống mắt giống nhau hoàn toàn là gần như không thể.




![eye](image/humaniris.jpg)
![giando](image/giando.png)

# II. Eye-tracker với OpenCV

- Eye-tracking là một công nghệ ứng dụng trong rất nhiều bài toán, và như kể trên, Eye-tracker là cơ sở cho nhiều bài toán đó, việc xác định vị trí mống mắt là rất quan trọng để làm tiền đề cho các bước tiếp theo

- Những bài toán tiêu biểu với mống mắt có thể kể đến: Eye-tracking, Iris Recognition, 

## 1. Giới thiệu về OpenCV

- OpenCV (Open Computer Vision) là một thư viện mã nguồn mở hàng đầu cho xử lý về thị giác máy tính, machine learning, xử lý ảnh. OpenCV đươc viết bằng C/C++, vì vậy có tốc độ tính toán rất nhanh, có thể sử dụng với các ứng dụng liên quan đến thời gian thực. Opencv có các interface cho C/C++, Python Java vì vậy hỗ trợ được cho Window, Linux, MacOs lẫn Android, iOS OpenCV có cộng đồng hơn 47 nghìn người dùng và số lượng download vượt quá 6 triệu lần

## 2. Ý tưởng cho bài toán xác Eye-tracker bằng OpenCV

## 3. Simple Eye-tracker với OpenCV

In [ ]:
!python3

In [6]:
import cv2
import numpy

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

ModuleNotFoundError: No module named 'cv2'